<a href="https://colab.research.google.com/github/BhaavyaShukla/BhaavyaShukla/blob/main/i_aoma_5dofs_11_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intelligent-driven Automatic Operational Modal Analysis i-AOMA
Marco Martino Rosso, Angelo Aloisio, Giuseppe Carlo Marano, and Giuseppe Quaranta

This code is running Monte Carlo (MC) Stochastic Subspace Identification covariance-based (SSI-cov)

Created on March 6 2023
@author: Marco Martino Rosso

# Get code from dropbox or github repository and install modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %cd /content/drive/MyDrive/Roydon_task
# !wget -O iAOMA_v_1_0.zip https://www.dropbox.com/s/a4six4v61b2t33s/iAOMA_v_1_0.zip?dl=0
# !unzip "iAOMA_v_1_0.zip" -d /content/drive/MyDrive/Roydon_task/code_decision_tree

/content/drive/MyDrive/Roydon_task
--2024-02-24 01:53:00--  https://www.dropbox.com/s/a4six4v61b2t33s/iAOMA_v_1_0.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/a4six4v61b2t33s/iAOMA_v_1_0.zip [following]
--2024-02-24 01:53:01--  https://www.dropbox.com/s/raw/a4six4v61b2t33s/iAOMA_v_1_0.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca5ab601e74c2843728f2415256.dl-eu.dropboxusercontent.com/cd/0/inline/CN0b6kzDPGsaYh7zH8V_Lpl9DffHG-1gw4lhV2DfPG_qlGGrIl_ui5tTDfZE9jMp4rOsVPovUqz4Kh4Qm-BBqVzmfVN0MonkiKi5dehGM7ZLURsEERgiza54SbQG7TOuEUuCUmeSiBrvf3_XHXrQmShX/file# [following]
--2024-02-24 01:53:01--  https://uca5ab601e74c2843728f2415256.dl-eu.dropboxusercontent.com/cd/0/inline/CN0b6kzDPGsaYh7zH8V_Lpl9DffHG-1gw4lhV2DfPG_qlGG

In [ ]:
%cd /content/drive/MyDrive/Roydon_task/code_decision_tree

/content/drive/MyDrive/Roydon_task/code_decision_tree


# Run i-AOMA

## Import modules and classes

In [ ]:
"""
Intelligent-driven Automatic Operational Modal Analysis i-AOMA
Marco Martino Rosso, Angelo Aloisio, Giuseppe Carlo Marano, and Giuseppe Quaranta

This code is running Monte Carlo (MC) Stochastic Subspace Identification covariance-based (SSI-cov)

Created on March 6 2023
@author: Marco Martino Rosso
"""
# %% Import modules
!pip install KDEpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
plt.rcParams['font.family'] = 'serif'


from ssicov.ssi import SsiCov
from ssicov.SDres import SdRes
from datapreprocessing.helpers import *
from user_definitions import *
from utilities.utils import *
from qmc.qmc_sampling import QmcSampler
from kde.kdepy import Kde
from rfcore.RFcore import RfCore
from convergencecheck.convcheck import ConvCheck
from plotting_fns.plot_mode_shapes import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.4/553.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Paths definitions
DATA_FILE = 'DATA/5DOF_fixed_ex1.txt'
RESULTS_PATH = 'RESULTS_decisiontree'
N_DIM = 3 # Problem dimensions, accepted integers 2 or 3 for plotting mode shapes in 2D or 3D structures
NODESFILE = 'DATA/nodes1.txt'
CONNECTIVITYFILE = 'DATA/connectivity1.txt'
MONITORED_DOF_FILE = 'DATA/nodes_mode_shape_dofs1.txt'

# Vibration data info
## [Hz]
SAMPLING_FREQUENCY = 40
DECIMATION_FACTOR = 1
FUNDAMENTAL_FREQUENCY = 2
DATAFILTERING = False
BUTTERWORTH_DESIGN = [5,30,'highpass'] # [N,Wn,btype] N:order of the filter; Wn: cut-off frequency; btype: ‘lowpass’, ‘highpass’, ‘bandpass’, ‘bandstop’

# Monte Carlo settings:
MAX_NUM_MC_SIM_PHASE_1 = 100
MAX_NUM_MC_SIM_PHASE_2 = 1000

# Mode shape normalization
# accepted: integer of specific dof otherwise a string 'max'
NORMALIZDOF = 0 #'max'

# KDE prominence peaks
# Prominence may be an arbitrary float between 0 and 1 (suggested 0.1)
# otherwise the string 'automatic'
KDEPROMINENCE = 0.05 # 0.1

# RF intelligent core information content IC threshold
ICTHRESH = 0.1

# i-AOMA PHASE 2
BATCHNSIM = 50         # evaluate the convergence every 50 analyses
CONVMCTHRESH = 0.02    # track relative differences covariance matrix within +-2% for
                       # acceptable shifting convergence band rule (ASCBR)

# Plotting flags
PLOTSIGNALS = True
PLOTSVDOFPSD = True
SAVEPLOTSIGNALS = True
SAVEPLOTSVDOFPSD = True

PLOT_OVERLAPPED_SD = True

# Plotting options
# Standard deviation factor
MODESCALEFCT = 1
MODESTDFCT = 3
MODESTDFCT_ELLIPSES = 3 # it is accounted only for 3D problems
MODE2D_DIRECTION = 'vertical' # it is accounted only for 2D problems, accepted 'vertical' or 'horizontal'

## i-AOMA phase 1 exploratory

In [ ]:
# from utilities.utils import *
import numpy

# Import data and decimate
create_results_folder(RESULTS_PATH+'/Phase1')
data = import_data(DATA_FILE)


data, fs = datadecimate(data, fs=SAMPLING_FREQUENCY, q=DECIMATION_FACTOR) #fs: [Hz] Decimated sampling frequency

if DATAFILTERING:
    data = datafiltering(data, fs, BUTTERWORTH_DESIGN)

visualize_plot_signals(data, PLOTSIGNALS, RESULTS_PATH+'/Phase1', fs, SAVEPLOTSIGNALS, labelsformat=['Time [s]','Acceleration [mm/s^2]'])

data1 = []
for i in range(5):
  for j in range(data.shape[0]):
    data1.append(data[j])

data1 = numpy.asarray(data1)

svPSD = visualize_plot_PSD(data1, PLOTSVDOFPSD, RESULTS_PATH+'/Phase1', fs, SAVEPLOTSVDOFPSD)

# i-AOMA Phase 1
print('\n\ni-AOMA phase 1: EXPLORATIVE MC SSI-cov SIMULATIONS\n')
Qmc = QmcSampler(data, FUNDAMENTAL_FREQUENCY, fs)

count_sim = -1
count_sim_effective = -1
discardedpar=[]
discardedpar_errtype=[]
ok_counter = 1

# admissiblepar = []
# stabdiag_columns = ['Frequency', 'Order', 'Label', 'Damp', 'Emme', 'ModeNum', 'SimNumber']
# stabdiag = np.array([])
# stabdiag_modes = np.array([]) # first element is the 'SimNumber' the remaining columns are referred to the dofs

while count_sim_effective < (MAX_NUM_MC_SIM_PHASE_1-1):
    try:
        count_sim += 1
        sampledpar = Qmc.Halton() # it samples an array of [time shift, max order, window lenght, time target]
        # Run SSI-cov for a set of QMC Halton parameters with timeout of 30 seconds
        ssicovSD = SsiCov(dataslice(data, *sampledpar[0][2:]), fs, *sampledpar[0][:2])
        # If achieve here without any error, a new effective simulation is accounted
        count_sim_effective +=1
        # Normalize mode shapes
        ssicovSD.mode_normalization(NORMALIZDOF)
        # Retain only stable poles and associated mode shapes, convert the results in numpy arrays
        ssicovSD.retain_stable_poles(sim_num=count_sim_effective)

        if count_sim_effective == 0:
            # Initialize the Stabilization Diagram results collector object
            SDresults = SdRes(ssicovSD = ssicovSD, admissiblepar = np.array(Qmc.retrieve_last_qmc_sampled()[0]) )
        else: # Update the Stabilization Diagram results collector object with new results
            SDresults.update_results(ssicovSD = ssicovSD, admissiblepar = np.array(Qmc.retrieve_last_qmc_sampled()[0]) )

        print(f'Simulation {count_sim} status: {ok_counter}th OK')
        ok_counter += 1

    except Exception as e:
        if 'could not broadcast input array from shape' in str(e):
            print(f'Simulation {count_sim} status: ErrorType'+ ' ' + 'Incompatible parameters set')
        else:
            print(f'Simulation {count_sim} status: ErrorType'+ ' ' + str(e))
        discardedpar.append(Qmc.retrieve_last_qmc_sampled()[0])
        discardedpar_errtype.append(str(e))






i-AOMA phase 1: EXPLORATIVE MC SSI-cov SIMULATIONS

Simulation 0 status: ErrorType Connection timed out
Simulation 1 status: ErrorType Connection timed out
Simulation 2 status: ErrorType Connection timed out
Simulation 3 status: ErrorType Connection timed out
Simulation 4 status: ErrorType Connection timed out
Simulation 5 status: ErrorType Connection timed out
Simulation 6 status: ErrorType Connection timed out
Simulation 7 status: ErrorType Connection timed out
Simulation 8 status: ErrorType Connection timed out
Simulation 9 status: ErrorType Connection timed out
Simulation 10 status: ErrorType Connection timed out
Simulation 11 status: ErrorType Connection timed out
Simulation 12 status: ErrorType Connection timed out
Simulation 13 status: ErrorType Connection timed out
Simulation 14 status: ErrorType Connection timed out
Simulation 15 status: ErrorType Connection timed out
Simulation 16 status: ErrorType Connection timed out
Simulation 17 status: 1th OK
Simulation 18 status: Erro

In [ ]:
print(f'\nSimulations evaluated {count_sim+1}, discarding {len(discardedpar)} parameters to collect {MAX_NUM_MC_SIM_PHASE_1} useful results')

SDresults.plot_overlapped_SD(fs,PLOT_OVERLAPPED_SD, RESULTS_PATH+'/Phase1')
SDresults.plot_overlapped_SD_stable(fs,PLOT_OVERLAPPED_SD, RESULTS_PATH+'/Phase1')
SDresults.export_results_to_file(RESULTS_PATH+'/Phase1')
#  create attribute which join in a single array poles and modes SDresults.jointpolesmodes
SDresults.jointpolesarray_and_modesarray() # SDresults.joint_col_names [ ['Frequency', 'Order', 'Label', 'Damp', 'Emme', 'ModeNum', 'SimNumber'], ['SimNumber','dof','dof','...'] ]

# KDE
kdeSD = Kde(SDresults.jointpolesmodes, fs, KDEPROMINENCE)
kdeSD.plot_kde_freq(RESULTS_PATH+'/Phase1')
kdeSD.select_modes_clusters()
# kdeSD.plot_select_modes_clusters(RESULTS_PATH+'/Phase1')
print(f'Found {kdeSD.peaksFFTKDE.shape[0]:d} peaks at {kdeSD.x[kdeSD.peaksFFTKDE]} Hz with prominence of {kdeSD.KDEPROMINENCE:.4f}.\n')
print(f'Found {len(kdeSD.Frequency_dataset):d} poles clusters within {kdeSD.KDEbwFactor:d} times the bandwidth of {kdeSD.bw:.4f} Hz.\n')
# IC
IC = kdeSD.information_content(SDresults.selectedpoles_totnum)
kdeSD.save_plot_IC(MAX_NUM_MC_SIM_PHASE_1-1, ICTHRESH, RESULTS_PATH+'/Phase1')
kdeSD.export_results_to_file(RESULTS_PATH+'/Phase1')
# RF
RF = RfCore(ICTHRESH, IC, SDresults.admissiblepar, discardedpar)
RF.fit()
RF.save_model_to_file(RESULTS_PATH+'/Phase1')
RF.evaluate(RF.x, RF.y)


Simulations evaluated 585, discarding 485 parameters to collect 100 useful results
All cluster are full with 1 times of the bandwidth of 0.0038
Found 19 peaks at [ 4.23227164  4.52652313  4.88157936  4.95324209  5.74044627  5.9988664
  6.37238121  8.40391089  8.4864316   8.65798782  8.86103221  9.65257958
 10.79918319 11.61461814 12.79162411 14.03812121 14.39752063 15.20969818
 19.25212736] Hz with prominence of 0.0500.

Found 19 poles clusters within 1 times the bandwidth of 0.0038 Hz.

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       568
         1.0       0.74      0.82      0.78        17

    accuracy                           0.99       585
   macro avg       0.87      0.91      0.89       585
weighted avg       0.99      0.99      0.99       585

Confusion Matrix:
[[563   5]
 [  3  14]]


In [ ]:
# !zip -r /content/RESULTS_phase1.zip /content/drive/MyDrive/Roydon_task/code_decision_tree/RESULTS
# from google.colab import files
# files.download("/content/RESULTS_phase1.zip")

## i-AOMA phase 2 intelligent MC SSI-cov

In [ ]:
# i-AOMA Phase 2 che parte dai risultati già immagazzinati della fase 1
print('\n\ni-AOMA phase 2: INTELLIGENT Decision Tree DRIVEN MC SSI-cov SIMULATIONS\n')
starting_sim_num = [count_sim, len(discardedpar), count_sim_effective]
continueMCsim=1
# Initialize convergence check object which stores uncertainty propagation results
modesconv = ConvCheck(kdeSD.Frequency_dataset, MAX_NUM_MC_SIM_PHASE_1)
last_check_sim = count_sim_effective

create_results_folder(RESULTS_PATH+'/Phase2')
create_results_folder(RESULTS_PATH+'/Phase2/Backup_convergence_checks') #???? not implemented for now because saving results will slow down the code

while continueMCsim and (count_sim_effective < (MAX_NUM_MC_SIM_PHASE_1 + MAX_NUM_MC_SIM_PHASE_2 -1)) :
    try:
        count_sim += 1
        sampledpar = Qmc.Halton() # it samples an array of [time shift, max order, window lenght, time target]
        y_pred = RF.predict( np.array(Qmc.retrieve_last_qmc_sampled()[0]) )
        print(y_pred)
        if y_pred == 1:
            print(f'Simulation {count_sim} status: Accepted by Decision tree')
            # Run SSI-cov for a set of QMC Halton parameters with timeout of 30 seconds
            ssicovSD = SsiCov(dataslice(data, *sampledpar[0][2:]), fs, *sampledpar[0][:2])
            # If achieve here without any error, a new effective simulation is accounted
            count_sim_effective +=1
            # Normalize mode shapes
            ssicovSD.mode_normalization(NORMALIZDOF)
            # Retain only stable poles and associated mode shapes, convert the results in numpy arrays
            ssicovSD.retain_stable_poles(sim_num=count_sim_effective)

            SDresults.update_results(ssicovSD = ssicovSD, admissiblepar = np.array(Qmc.retrieve_last_qmc_sampled()[0]) )

            print(f'Simulation {count_sim} status: OK')
        else:
            print(f'Simulation {count_sim} status: Discarded by Decision tree')
            discardedpar.append(Qmc.retrieve_last_qmc_sampled()[0])
            discardedpar_errtype.append(f'Decision tree prediction: 0')

    except Exception as e:
        if 'could not broadcast input array from shape' in str(e):
            print(f'Simulation {count_sim} status: ErrorType'+ ' ' + 'Incompatible parameters set')
        else:
            print(f'Simulation {count_sim} status: ErrorType'+ ' ' + str(e))
        discardedpar.append(Qmc.retrieve_last_qmc_sampled()[0])
        discardedpar_errtype.append(str(e))

    if ((count_sim_effective - starting_sim_num[2]) % BATCHNSIM == 0) and (count_sim_effective - starting_sim_num[2])!=0 \
        and (last_check_sim != count_sim_effective) :
        print('********** Check convergence criteria ********** \n')
        print(f'Actual simulations so far: {count_sim_effective+1}')

        create_results_folder(RESULTS_PATH+f'/Phase2/Backup_convergence_checks/{count_sim_effective+1}')


        #  create attribute which join in a single array poles and modes SDresults.jointpolesmodes
        SDresults.jointpolesarray_and_modesarray() # SDresults.joint_col_names [ ['Frequency', 'Order', 'Label', 'Damp', 'Emme', 'ModeNum', 'SimNumber'], ['SimNumber','dof','dof','...'] ]
        kdeSD.select_peaks_phase_two(SDresults.jointpolesmodes)
        kdeSD.plot_kde_freq(RESULTS_PATH+f'/Phase2/Backup_convergence_checks/{count_sim_effective+1}') # comment it after checking everything is correct
        kdeSD.select_modes_clusters()
        # kdeSD.plot_select_modes_clusters(RESULTS_PATH+f'/Phase2/Backup_convergence_checks/{count_sim_effective+1}') # comment it after checking everything is correct

        convergence_reached = modesconv.update_converg_sim(kdeSD.Frequency_dataset, count_sim_effective, last_check_sim, CONVMCTHRESH)

        modesconv.plot_convergence_curves(CONVMCTHRESH, RESULTS_PATH+f'/Phase2/Backup_convergence_checks/{count_sim_effective+1}') # comment it after checking everything is correct

        last_check_sim = count_sim_effective

        if convergence_reached:
            print('********** Stopping criteria reached! ********** \n\n')
            continueMCsim=0
        else:
            print('********** Convergence criteria not reached yet ********** \n\n')

    if count_sim_effective == (MAX_NUM_MC_SIM_PHASE_1 + MAX_NUM_MC_SIM_PHASE_2 -1):
        print('********** Convergence criteria not reached yet, but maximum allowed number of actual simulations reached ********** \n')

print(f'Simulations evaluated in total {count_sim+1}:\n' + \
      f'Phase 1: Simulations {starting_sim_num[0]+1}, discarding {starting_sim_num[1]}, collecting {MAX_NUM_MC_SIM_PHASE_1} useful results \n' + \
      f'Phase 2: Simulations {(count_sim-starting_sim_num[0])}, intelligently discarding {(len(discardedpar)-starting_sim_num[1])}, collecting {count_sim_effective-starting_sim_num[2]} useful results \n\n')

Streaming output truncated to the last 5000 lines.
0
Simulation 23320 status: Discarded by Decision tree
0
Simulation 23321 status: Discarded by Decision tree
0
Simulation 23322 status: Discarded by Decision tree
0
Simulation 23323 status: Discarded by Decision tree
0
Simulation 23324 status: Discarded by Decision tree
0
Simulation 23325 status: Discarded by Decision tree
0
Simulation 23326 status: Discarded by Decision tree
0
Simulation 23327 status: Discarded by Decision tree
0
Simulation 23328 status: Discarded by Decision tree
0
Simulation 23329 status: Discarded by Decision tree
0
Simulation 23330 status: Discarded by Decision tree
1
Simulation 23331 status: Accepted by Decision tree
Simulation 23331 status: OK
1
Simulation 23332 status: Accepted by Decision tree
Simulation 23332 status: ErrorType Incompatible parameters set
0
Simulation 23333 status: Discarded by Decision tree
0
Simulation 23334 status: Discarded by Decision tree
0
Simulation 23335 status: Discarded by Decision t

### Plotting and exporting results

In [ ]:
# Export results
RF.save_prediction_to_file(RESULTS_PATH+'/Phase2')
SDresults.plot_overlapped_SD(fs,PLOT_OVERLAPPED_SD, RESULTS_PATH+'/Phase2')
SDresults.plot_overlapped_SD_stable(fs,PLOT_OVERLAPPED_SD, RESULTS_PATH+'/Phase2')
SDresults.export_results_to_file(RESULTS_PATH+'/Phase2')
kdeSD.plot_kde_freq(RESULTS_PATH+'/Phase2')
# kdeSD.plot_select_modes_clusters(RESULTS_PATH+'/Phase2')
print(f'Found {kdeSD.peaksFFTKDE.shape[0]:d} peaks at {kdeSD.x[kdeSD.peaksFFTKDE]} Hz with prominence of {kdeSD.KDEPROMINENCE:.4f}.\n')
print(f'Found {len(kdeSD.Frequency_dataset):d} poles clusters within {kdeSD.KDEbwFactor:d} times the bandwidth of {kdeSD.bw:.4f} Hz.\n')
# IC
IC_final = kdeSD.information_content(SDresults.selectedpoles_totnum)
# kdeSD.save_plot_IC(count_sim_effective, ICTHRESH, RESULTS_PATH+'/Phase2')
kdeSD.export_results_to_file(RESULTS_PATH+'/Phase2')
modesconv.export_results_to_file(RESULTS_PATH+'/Phase2')

# Mode shape uncertainties analysis, saving and plotting final results
create_results_folder(RESULTS_PATH+'/Phase2/Mode_shapes')
create_results_folder(RESULTS_PATH+'/Phase2/Convergence_Analysis')
create_results_folder(RESULTS_PATH+'/Phase2/Intelligent_sampling_maps')

nodes = pd.read_csv(NODESFILE, header=None, sep="\s+", index_col=False).to_numpy()
connectivity = pd.read_csv(CONNECTIVITYFILE, header=None, sep="\s+", index_col=False).to_numpy()

# modesconv.plot_mode_shapes(N_DIM, nodes, connectivity, MODESCALEFCT, MODESTDFCT, MODESTDFCT_ELLIPSES, MODE2D_DIRECTION, RESULTS_PATH+'/Phase2/Mode_shapes')
modesconv.plot_convergence_curves(CONVMCTHRESH, RESULTS_PATH+'/Phase2/Convergence_Analysis')
SDresults.plot_sampling_maps(RESULTS_PATH+'/Phase2/Intelligent_sampling_maps')

Found 14 peaks at [ 4.22481117  4.52591585  4.87702347  4.95094086  5.74772688  5.99882862
  6.37711175  8.47940938  9.65991364 10.80454622 12.76770523 14.04495432
 15.19719604 19.2583046 ] Hz with prominence of 0.0500.

Found 14 poles clusters within 1 times the bandwidth of 0.0022 Hz.



# Sandbox

Adjust plot convergence analysis

In [ ]:
def plot_trace(modes_trace_covariance, modes_trace_covariance_rel_diff, freq_mean, CONVMCTHRESH, RESULTS_PATH):
    fig,ax=plt.subplots(figsize=(10,4),facecolor='white')
    fig2,ax2=plt.subplots(figsize=(10,4),facecolor='white')
    for jj in range(modes_trace_covariance.shape[1]):
        ax.plot(np.arange(1,modes_trace_covariance.shape[0]+1), modes_trace_covariance[:,jj],
                 label=f'Mode at {freq_mean[jj]:.2f} Hz', lw=3)
        ax2.plot(np.arange(2,modes_trace_covariance_rel_diff.shape[0]+2), modes_trace_covariance_rel_diff[:,jj],
                 label=f'Mode at {freq_mean[jj]:.2f} Hz', lw=1)
    # ax.xaxis.set_major_locator(mticker.MultipleLocator(1)); ax2.xaxis.set_major_locator(mticker.MultipleLocator(1))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%d')); ax2.xaxis.set_major_formatter(FormatStrFormatter('%d'))

    ax.legend(loc='upper left')
    # plt.legend(loc='upper right',bbox_to_anchor=(0.,0.,1.,0.9),ncol=4,framealpha=0.9)
    ax.set_title('Trace covariance matrix Convergence',fontweight='bold')
    ax.set_xlabel('Actually conducted simulations')
    ax.set_ylim([0,10])
    fig.tight_layout()

    ax2.plot(np.arange(2,modes_trace_covariance_rel_diff.shape[0]+2), CONVMCTHRESH*np.ones(modes_trace_covariance_rel_diff.shape[0]),
             'b--',label='Band $\pm$2%')
    ax2.plot(np.arange(2,modes_trace_covariance_rel_diff.shape[0]+2), -CONVMCTHRESH*np.ones(modes_trace_covariance_rel_diff.shape[0]),'b--')
    ax2.fill_between(np.arange(2,modes_trace_covariance_rel_diff.shape[0]+2), CONVMCTHRESH*np.ones(modes_trace_covariance_rel_diff.shape[0]),
                     -CONVMCTHRESH*np.ones(modes_trace_covariance_rel_diff.shape[0]), color='yellow', alpha=0.5)
    # plt.fill_betweenx([-0.1,0.1], 500, 550, color='#63C7B2',alpha=0.5)
    # plt.text(380, -0.045, 'Convergence ASCBR', fontsize=12,color='#63C7B2',fontweight='bold')
    ax2.set_ylim(-0.1,0.1)
    ax2.legend(loc='upper left')
    ax2.set_title('Relative difference on trace covariance matrix Convergence',fontweight='bold')
    ax2.set_xlabel('Actually conducted simulations')
    fig2.tight_layout()

    fig.savefig(RESULTS_PATH + f'/Trace_covariance_matrix_Convergence_zoom.png', format='png', dpi=300)
    fig2.savefig(RESULTS_PATH + f'/Relative_difference_on_trace_covariance_matrix.png', format='png', dpi=300)

    plt.close('all')

In [ ]:
plot_trace(modesconv.modes_trace_covariance, modesconv.modes_trace_covariance_rel_diff, modesconv.freq_mean[-1], CONVMCTHRESH, RESULTS_PATH+'/Phase2/Convergence_Analysis')

Change std factor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter


def plotModeShape2D(_nodes, _connectivity, ax, color='black',style='solid',annotFlag=False):
    num_frames = _connectivity.shape[0]
    for k in range(num_frames):
        x1 = _nodes[_connectivity[k, 0] - 1, 0]
        y1 = _nodes[_connectivity[k, 0] - 1, 1]
        x2 = _nodes[_connectivity[k, 1] - 1, 0]
        y2 = _nodes[_connectivity[k, 1] - 1, 1]
        xx = [x1, x2]; yy = [y1, y2]
        ax.plot(xx, yy, color=color,linestyle=style,zorder=10)
    for i in range(_nodes.shape[0]):
        xs = _nodes[i, 0]; ys = _nodes[i, 1]
        ax.scatter(xs, ys, color=color, marker='o',zorder=10)
        if annotFlag:
            ax.annotate(f'P{i + 1}', xy=(xs, ys), xytext=(3, 3),
                        textcoords='offset points',zorder=10)

def plot_modes_for_2d(modes_mean, modes_std, freq_mean, _nodes, _connectivity, MODESCALEFCT, MODESTDFCT, MODE2D_DIRECTION, RESULTS_PATH):
    for ii in range(modes_mean.shape[0]):
        fig,ax = plt.subplots(figsize=(6,5),facecolor='white')
        plotModeShape2D(_nodes, _connectivity, ax, color='black', style='dashed', annotFlag=True)
        ax.set_xlabel('x [m]')
        ax.set_ylabel('z [m]')
        nodes_new = np.copy(_nodes)
        if MODE2D_DIRECTION == 'vertical' :
            nodes_new[1:,0] += MODESCALEFCT * modes_mean[ii,:]
            nodes_new_infStd = np.copy(nodes_new)
            nodes_new_supStd = np.copy(nodes_new)
            nodes_new_infStd[1:,0] -= MODESTDFCT * modes_std[ii,:]
            nodes_new_supStd[1:,0] += MODESTDFCT * modes_std[ii,:]
            plotModeShape2D(nodes_new, _connectivity, ax, color='#023e7d', annotFlag=False)
            ax.fill_betweenx(nodes_new[:,1],nodes_new_infStd[:,0],nodes_new_supStd[:,0],
                            color='#9BD0D1',
                            zorder=1,
                            label=f'{MODESTDFCT:.1f} Std. dev.')
        elif MODE2D_DIRECTION == 'horizontal':
            nodes_new[:,1] += MODESCALEFCT * modes_mean[ii,:]
            nodes_new_infStd = np.copy(nodes_new)
            nodes_new_supStd = np.copy(nodes_new)
            nodes_new_infStd[:,1] -= MODESTDFCT * modes_std[ii,:]
            nodes_new_supStd[:,1] += MODESTDFCT * modes_std[ii,:]
            plotModeShape2D(nodes_new, _connectivity, ax, color='#023e7d', annotFlag=False)
            ax.fill_between(nodes_new[:,0],nodes_new_infStd[:,1],nodes_new_supStd[:,1],
                            color='#9BD0D1',
                            zorder=1,
                            label=f'{MODESTDFCT:.1f} Std. dev.')
        else:
            raise ValueError("Error in direction of mode shapes. Accepted arguments: 'horizontal' or 'vertical' ")

        plt.title(f'Mode shape at {freq_mean[ii]:.3f} Hz', fontweight='bold')
        custom_lines = [
                    Line2D([0], [0], color='black',linestyle='dashed', lw=4),
                    Line2D([0], [0], color='#023e7d', lw=4),
                    Patch(facecolor='#9BD0D1', edgecolor=None)
                    ]
        plt.legend(custom_lines,['Undeformed shape','Deformed shape',f'{MODESTDFCT:.1f} Std. dev.'],loc='best')
        plt.ylim(plt.gca().get_ylim()[0],plt.gca().get_ylim()[1]+0.5)
        plt.tight_layout()
        plt.savefig(RESULTS_PATH + f'/Mode_shape_{ii+1}.png', format='png', dpi=300)
        plt.close()

def plot_mode_shapes(N_DIM, nodes, connectivity, MODESCALEFCT, MODESTDFCT, MODESTDFCT_ELLIPSES, MODE2D_DIRECTION, RESULTS_PATH, modesconv):
    if N_DIM == 2:
        plot_modes_for_2d(modesconv.modes_mean[-1], modesconv.modes_std[-1], modesconv.freq_mean[-1], nodes, \
                          connectivity, MODESCALEFCT, MODESTDFCT, MODE2D_DIRECTION, RESULTS_PATH)
        print("2d")
    elif N_DIM == 3: # N_DIM == 3
        plot_modes_for_3d()
    else:
        raise ValueError("Unrecognized Number of dimensions to plot mode shapes")


In [ ]:
# MODESTDFCT=0.1
# N_DIM = 2

In [ ]:
# plot_mode_shapes(N_DIM, nodes, connectivity, MODESCALEFCT, MODESTDFCT, MODESTDFCT_ELLIPSES, MODE2D_DIRECTION, RESULTS_PATH+'/Phase2/Mode_shapes', modesconv)

In [ ]:
!zip -r /content/RESULTS.zip /content/drive/MyDrive/Roydon_task/code_decision_tree/RESULTS
from google.colab import files
files.download("/content/RESULTS.zip")